In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/GOPRO_Large.zip" -d "/content/datasets"

In [ ]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

def transform_sharp(image):
    # Provides a 720x1280 image for sharp_img
    return transforms.Compose([
        transforms.Resize((720, 1280)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])(image)

def transform_blur(image):
    # Provide a 360x640 image to blur_img
    return transforms.Compose([
        transforms.Resize((360, 640)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])(image)

class CustomDataset(Dataset):
    def __init__(self, root_dir, mix_ratio=0.5):
        self.root_dir = os.path.expanduser(root_dir)
        self.mix_ratio = mix_ratio
        self.samples = self._load_samples()

    def _load_samples(self):
        samples = []
        for subdir in sorted(os.listdir(self.root_dir)):
            subdir_path = os.path.join(self.root_dir, subdir)
            if os.path.isdir(subdir_path):
                sharp_imgs = sorted(os.listdir(os.path.join(subdir_path, 'sharp')))
                blur_imgs = sorted(os.listdir(os.path.join(subdir_path, 'blur')))
                blur_gamma_imgs = sorted(os.listdir(os.path.join(subdir_path, 'blur_gamma')))

                for sharp_img, blur_img, blur_gamma_img in zip(sharp_imgs, blur_imgs, blur_gamma_imgs):
                    sharp_path = os.path.join(subdir_path, 'sharp', sharp_img)
                    blur_path = os.path.join(subdir_path, 'blur', blur_img)
                    blur_gamma_path = os.path.join(subdir_path, 'blur_gamma', blur_gamma_img)
                    samples.append((sharp_path, blur_path, blur_gamma_path))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sharp_path, blur_path, blur_gamma_path = self.samples[idx]
        sharp_img = Image.open(sharp_path).convert('RGB')
        blur_img = Image.open(blur_path).convert('RGB')
        blur_gamma_img = Image.open(blur_gamma_path).convert('RGB')

        # Mix blur and blur_gamma images
        blur_img = Image.blend(blur_img, blur_gamma_img, self.mix_ratio)
        sharp_img = transform_sharp(sharp_img)
        blur_img = transform_blur(blur_img)

        return blur_img, sharp_img

# Function used to save model output
def save_model_output(output, filename):
    output = output.cpu().detach()
    output_img = transforms.ToPILImage()(output).convert('RGB')
    output_img.save(filename)

train_dataset = CustomDataset(root_dir='/content/datasets/train')
# Split the data set into training set and validation set
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(
    train_subset,
    batch_size=2,
    shuffle=True,
    num_workers=8
)
val_loader = DataLoader(
    val_subset,
    batch_size=2,
    shuffle=False,
    num_workers=8
)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.distributed as distance
from torch.cuda.amp import GradScaler, autocast

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#def Perceptual Loss by MobileNet
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        self.mobilenet = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT).features
        self.mobilenet.eval()
        for param in self.mobilenet.parameters():
            param.requires_grad = False

    def forward(self, input, target):
        input_features = self.mobilenet(input)
        target_features = self.mobilenet(target)
        # Resize if necessary
        if input_features.shape[2:] != target_features.shape[2:]:
            input_features = F.interpolate(input_features, size=target_features.shape[2:], mode='bilinear', align_corners=False)

        loss = nn.functional.mse_loss(input_features, target_features)
        return loss

In [ ]:
#Define the DnCNN Model
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []

        # Initial convolution layer
        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
        layers.append(nn.ReLU(inplace=True))

        # Middle layers
        for _ in range(num_of_layers - 2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))

        # Final convolution layer
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))

        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        out = self.dncnn(x)
        return out

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
# Definite U-Net
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256)
        self.up2 = Up(512, 128)
        self.up3 = Up(256, 64)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.relu(residual)
        residual = self.conv2(residual)
        return x + residual

# EDSR Model, use scale_factor to choose scale
class EDSR(nn.Module):
    def __init__(self, scale_factor=2, num_channels=3, num_residual_blocks=16):
        super(EDSR, self).__init__()
        self.num_channels = num_channels

        # First layer
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=4)

        # Residual blocks
        self.residual_blocks = nn.Sequential(*[ResidualBlock(64) for _ in range(num_residual_blocks)])

        # Second conv layer post residual blocks
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        # Upsampling layers
        self.upsampling = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(upscale_factor=scale_factor),
            nn.ReLU(inplace=True)
        )

        # Output layer
        self.conv3 = nn.Conv2d(64, num_channels, kernel_size=9, padding=4)

    def forward(self, x):
        out = self.conv1(x)
        residual = out
        out = self.residual_blocks(out)
        out = self.conv2(out)
        out = out + residual  # Element-wise sum
        out = self.upsampling(out)
        out = self.conv3(out)
        return out

#Create EDSR model instance
model = EDSR(scale_factor=2, num_channels=3, num_residual_blocks=16)

In [ ]:
# Add SobelEdgeLoss
class SobelEdgeLoss(nn.Module):
    def __init__(self):
        super(SobelEdgeLoss, self).__init__()
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3)
        self.sobel_x = sobel_x.repeat(3, 1, 1, 1).to(device)
        self.sobel_y = sobel_y.repeat(3, 1, 1, 1).to(device)

    def forward(self, input, target):
        input_x = F.conv2d(input, self.sobel_x, padding=1, groups=3)
        input_y = F.conv2d(input, self.sobel_y, padding=1, groups=3)
        target_x = F.conv2d(target, self.sobel_x, padding=1, groups=3)
        target_y = F.conv2d(target, self.sobel_y, padding=1, groups=3)

        input_edge = torch.sqrt(input_x ** 2 + input_y ** 2)
        target_edge = torch.sqrt(target_x ** 2 + target_y ** 2)

        loss = F.mse_loss(input_edge, target_edge)
        return loss

In [ ]:
dncnn = DnCNN(channels=3).to(device)
unet = UNet(n_channels=3, n_classes=3).to(device)
edsr = EDSR(scale_factor=2, num_channels=3, num_residual_blocks=16)

In [ ]:
 dncnn.load_state_dict(torch.load('/content/drive/MyDrive/model/DnCNN/denoising_DnCNN3.pth'))
 unet.load_state_dict(torch.load('/content/drive/MyDrive/model/UNet/deblured_UNet4.pth'))
 edsr.load_state_dict(torch.load('/content/drive/MyDrive/model/EDSR/SuperResolution_EDSR.pth'))

In [ ]:
class JointModel(nn.Module):
    def __init__(self, dncnn, unet, edsr):
        super(JointModel, self).__init__()
        self.dncnn = dncnn
        self.unet = unet
        self.edsr = edsr

    def forward(self, x):
        x = self.dncnn(x)  # denoised
        x = self.unet(x)  # deblured
        x = self.edsr(x)  # super resolution
        return x

joint_model = JointModel(dncnn, unet, edsr).to(device)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Creat model and Adam optimizer
mse_criterion = nn.MSELoss()
perceptual_criterion = PerceptualLoss().to(device)
optimizer = torch.optim.Adam(joint_model.parameters(), lr=0.00001)

# Initialize the ReduceLROnPlateau scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5, verbose=True)

In [ ]:
#Defining Loss Function Weights
mse_weight = 1.0
perceptual_weight = 0.1

In [ ]:
# Train model
def train_model(joint_model, train_loader, val_loader, mse_criterion, perceptual_criterion, optimizer, num_epochs=300, early_stopping_tolerance=15):
    best_val_loss = float('inf')
    no_improvement_count = 0  # Early stopping counter

    sobel_criterion = SobelEdgeLoss().to(device)
    sobel_weight = 0.05

    scaler = GradScaler()
    for epoch in range(num_epochs):
        joint_model.train()
        running_loss = 0.0

        for blur_img, transformed_sharp_img in train_loader:
            blur_img = blur_img.to(device)
            transformed_sharp_img = transformed_sharp_img.to(device)

            optimizer.zero_grad()

           # Performing forward propagation using the autocast context
            with autocast():
                outputs = joint_model(blur_img)
                mse_loss = mse_criterion(outputs, transformed_sharp_img)
                perceptual_loss = perceptual_criterion(outputs, transformed_sharp_img)
                sobel_loss = sobel_criterion(outputs, transformed_sharp_img)
                total_loss = mse_weight * mse_loss + perceptual_weight * perceptual_loss + sobel_weight * sobel_loss

            # Performing backward propagation and optimization using GradScaler
            scaler.scale(total_loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += total_loss.item() * blur_img.size(0)
        train_loss = running_loss / len(train_loader.dataset)

        # Validation test
        joint_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for blur_img, transformed_sharp_img in val_loader:
                blur_img = blur_img.to(device)
                transformed_sharp_img = transformed_sharp_img.to(device)
                outputs = joint_model(blur_img)
                mse_loss = mse_criterion(outputs, transformed_sharp_img)
                perceptual_loss = perceptual_criterion(outputs, transformed_sharp_img)
                sobel_loss = sobel_criterion(outputs, transformed_sharp_img)
                total_loss = mse_weight * mse_loss + perceptual_weight * perceptual_loss + sobel_weight * sobel_loss
                val_loss += total_loss.item() * blur_img.size(0)

        val_loss /= len(val_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

        scheduler.step(val_loss)
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            if no_improvement_count >= early_stopping_tolerance:
                print("Stopping early due to no improvement in validation loss")
                break

    return joint_model

train_model(joint_model, train_loader, val_loader, mse_criterion, perceptual_criterion, optimizer, num_epochs=300, early_stopping_tolerance=8)

In [ ]:
# Save model
if torch.cuda.is_available() and torch.cuda.current_device() == 0:
    model_path = '/content/drive/MyDrive/model/joint_model.pth'
    model_dir = os.path.expanduser(os.path.dirname(model_path))

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    torch.save(joint_model.state_dict(), os.path.expanduser(model_path))
    print(f"Model saved to {model_path}.")

Model saved to /content/drive/MyDrive/model/joint_model.pth.


In [ ]:
# Inverse normalization transformation
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
from skimage import img_as_float
import torch

# Initialize the sum of PSNR and SSIM
total_psnr_sharp = 0
total_ssim_sharp = 0
total_psnr_blur = 0
total_ssim_blur = 0
num_images = 0

with torch.no_grad():
    for blur_img, sharp_img in val_loader:
        blur_img = blur_img.to(device)
        sharp_img = sharp_img.to(device)

        deblured_img = joint_model(blur_img)

# Traverse the image to calculate PSNR and SSIM
        for i in range(blur_img.size(0)):
            deblured = inv_normalize(deblured_img[i]).clamp(0, 1)
            sharp = inv_normalize(sharp_img[i]).clamp(0, 1)
            blur = inv_normalize(blur_img[i]).clamp(0, 1)

            deblured_np = deblured.cpu().numpy().transpose(1, 2, 0)
            sharp_np = sharp.cpu().numpy().transpose(1, 2, 0)
            blur_np = blur.cpu().numpy().transpose(1, 2, 0)

    # Calculate PSNR and SSIM
            psnr_sharp = compare_psnr(deblured_np, sharp_np)
            ssim_sharp = compare_ssim(deblured_np, sharp_np, multichannel=True)

            total_psnr_sharp += psnr_sharp
            total_ssim_sharp += ssim_sharp
            num_images += 1

# Calculate average PSNR and SSIM
avg_psnr_sharp = total_psnr_sharp / num_images
avg_ssim_sharp = total_ssim_sharp / num_images


print(f'Average PSNR (Deblured vs Sharp): {avg_psnr_sharp}')
print(f'Average SSIM (Deblured vs Sharp): {avg_ssim_sharp}')

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os
import torch
from torchvision import transforms, utils
from PIL import Image
import torchvision

def process_blur_images(joint_model_path, dataset_root, output_folder):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    joint_model.load_state_dict(torch.load(joint_model_path, map_location=device))
    joint_model.eval()
    joint_model.to(device)

    os.makedirs(output_folder, exist_ok=True)

    img_counter = 1
    for subdir in sorted(os.listdir(dataset_root)):
        blur_folder = os.path.join(dataset_root, subdir, 'blur')
        if os.path.isdir(blur_folder):
            for blur_img_name in sorted(os.listdir(blur_folder)):
                blur_img_path = os.path.join(blur_folder, blur_img_name)
                blur_img = Image.open(blur_img_path).convert('RGB')
                blur_img_tensor = transform_blur(blur_img).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = joint_model(blur_img_tensor)

                output = inv_normalize(output.squeeze(0)).cpu()

                output_filename = f'img_{img_counter:04d}.png'
                output_image_path = os.path.join(output_folder, output_filename)
                utils.save_image(output, output_image_path)

                img_counter += 1

process_blur_images('/content/drive/MyDrive/model/joint_model.pth', '/content/datasets/train', '/content/image/processed_images')


In [ ]:
from PIL import Image, ImageFilter
import os

def apply_unsharp_mask_and_save(input_folder, output_folder, radius=2, percent=150, threshold=3):
    # Create the output folder if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # Process and save each image
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        with Image.open(img_path) as img:
            # Apply Unsharp Mask
            sharpened_img = img.filter(ImageFilter.UnsharpMask(radius=radius, percent=percent, threshold=threshold))

            # Save the sharpened image
            output_img_path = os.path.join(output_folder, img_name)
            sharpened_img.save(output_img_path)

# Use the function
apply_unsharp_mask_and_save('/content/image/processed_images', '/content/image/sharpened_images')


In [ ]:
import shutil
source_folder = '/content/image'
zip_file_path = '/content/drive/MyDrive/image.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', source_folder)